## Data Proceesing of Toronto Data

In [1]:
# Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium 
import json
import requests
from geopy.geocoders import Nominatim
%matplotlib inline

In [2]:
Tor_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
Tor_df = Tor_data[0]
Tor_df.head(5)

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park / Ontario Provincial Government,M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


In [28]:
TOR_LIST=[]
for i in range(len(Tor_df.index)):
    for j in range(len(Tor_df.columns)):
        try:
            pc,neighbour = Tor_df.iloc[i,j].split('(')  ## split the data into postal code and Neighbour
            postcode=pc[0:3]                            ## The frist 3 characters are the actual postal code
            l = len(pc)                                 ## Get the length of pc to extract the Borough
            borough=pc[3:l]                             ## extract the Borough 
#             print("Before split")
            nl = len(neighbour)                         ## Get the length of Neighbour to check if any "/"
            count = 0                                   ## Counter to check "/" in neighbour
#             print("Before split length of neighbour",nl)
            for n in range(nl):                         ## Loop to Check the neighbour value has "/"
                if neighbour[n] == '/':
#                     print("found  /")
                    count += 1
            if count == 0:                              ## If no counter then neighbour has only 1 occurence
#                 print(i,":",j,": ", neighbour)
                TOR_LIST.append([postcode,borough,neighbour])   ## Append the neighbour
            else:
#                 print("before split, inside else,",neighbour)    
                neighbour = neighbour.split("/")                ## Otherwise split the Neighbour with "/"
                #             print('After split len',i,j,":",len(neighbour))
                if len(neighbour)>1:
                    for n in range(len(neighbour)):
                        #                     print("n value",n,":",neighbour[n])
                        TOR_LIST.append([postcode,borough,neighbour[n]])
                else:
                    TOR_LIST.append([postcode,borough,neighbour])
        except:
            print("This is exception",i,":",j,Tor_df.iloc[i,j])
TOR_LIST[0:4]            

This is exception 0 : 0 M1ANot assigned
This is exception 0 : 1 M2ANot assigned
This is exception 0 : 6 M7AQueen's Park / Ontario Provincial Government
This is exception 0 : 7 M8ANot assigned
This is exception 1 : 1 M2BNot assigned
This is exception 1 : 6 M7BNot assigned
This is exception 1 : 7 M8BNot assigned
This is exception 2 : 1 M2CNot assigned
This is exception 2 : 2 M3CNorth York(Don Mills)South(Flemingdon Park)
This is exception 2 : 6 M7CNot assigned
This is exception 2 : 7 M8CNot assigned
This is exception 3 : 1 M2ENot assigned
This is exception 3 : 2 M3ENot assigned
This is exception 3 : 6 M7ENot assigned
This is exception 3 : 7 M8ENot assigned
This is exception 3 : 8 M9ENot assigned
This is exception 4 : 1 M2GNot assigned
This is exception 4 : 2 M3GNot assigned
This is exception 4 : 6 M7GNot assigned
This is exception 4 : 7 M8GNot assigned
This is exception 4 : 8 M9GNot assigned
This is exception 5 : 6 M7HNot assigned
This is exception 5 : 7 M8HNot assigned
This is exception

[['M3A', 'North York', 'Parkwoods)'],
 ['M4A', 'North York', 'Victoria Village)'],
 ['M5A', 'Downtown Toronto', 'Regent Park '],
 ['M5A', 'Downtown Toronto', ' Harbourfront)']]

In [29]:
X2 = [["M7A",'Downtown Toronto',"Queen's Park"],
["M3C","North York","Don Mills South"],
["M3C","North York","Flemingdon Park"], 
["M3C","North York","CFB Toronto"],
["M3K","North York","Downsview"]]

In [30]:
for n in range(len(X2)):
    TOR_LIST.append(X2[n])

In [31]:
# Remove the ")" from the Neighbourhood fields
for n in range(len(TOR_LIST)):
    if TOR_LIST[n][2][-1] == ')':
        TOR_LIST[n][2] = TOR_LIST[n][2][:-1]        

In [35]:
# df = pd.DataFrame(TOR_LIST,columns=['Postcode','Borough','Neighbourhood'])
TOR_LIST[205:215]

[['M8Y', 'Etobicoke', ' Royal York South East '],
 ['M8Y', 'Etobicoke', ' Kingsway Park South East'],
 ['M8Z', 'Etobicoke', 'Mimico NW '],
 ['M8Z', 'Etobicoke', ' The Queensway West '],
 ['M8Z', 'Etobicoke', ' South of Bloor '],
 ['M8Z', 'Etobicoke', ' Kingsway Park South West '],
 ['M8Z', 'Etobicoke', ' Royal York South West'],
 ['M7A', 'Downtown Toronto', "Queen's Park"],
 ['M3C', 'North York', 'Don Mills South'],
 ['M3C', 'North York', 'Flemingdon Park']]

In [36]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor


In [38]:
# read the geospatial data from the csv file http://cocl.us/Geospatial_data
url_TOR = 'http://cocl.us/Geospatial_data'
df_TOR_geo = pd.read_csv(url_TOR)
df_TOR_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [39]:
# Merge the Toroto dataset along with the geospatial dataset
DF_TOR = pd.merge(df,df_TOR_geo,left_on='Postcode',right_on='Postal Code',how='left')
DF_TOR.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
4,M6A,North York,Lawrence Manor,M6A,43.718518,-79.464763


In [40]:
DF_TOR = DF_TOR.drop(columns=['Postal Code'],axis=1)
DF_TOR.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


In [41]:
# Get the geospatial details of Toronto
address = 'Toronto, Toronto'
geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geospatial values of Toronto are {}:{}'.format(latitude,longitude))

The geospatial values of Toronto are 43.653963:-79.387207


In [42]:
## Use the folium to highlight the Neighbourhoods on world map
map_Tor = folium.Map(location=[latitude,longitude],zoom_start=10)

# Add markers on top of the Toronto Map
for Neighbour,Borough,lat,lng in zip(DF_TOR['Neighbourhood'],DF_TOR['Borough'],DF_TOR['Latitude'],DF_TOR['Longitude']):
    label=[f'{Neighbour},{Borough}']
    folium.CircleMarker(location=[lat,lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc').add_to(map_Tor)
   
map_Tor

In [43]:
# Prepare the credentials to call foursquare api
CLIENT_ID = '50UMKE0OQMLO2NQF0ERLPROW4J42AA2TTQ0SZJ2S230H3KIO'
CLIENT_SECRET = '5E0ML3XE2KKTLCAIQJVQLEKQ3NAFCYGW1U31FEQ3DRVPIHM2'
VERSION = '20200309'
RADIUS = 500
LIMIT = 100
url_api = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={DF_TOR.iloc[0,3]},{DF_TOR.iloc[0,4]}&radius={RADIUS}&limit={LIMIT}'
url_api

'https://api.foursquare.com/v2/venues/explore?&client_id=50UMKE0OQMLO2NQF0ERLPROW4J42AA2TTQ0SZJ2S230H3KIO&client_secret=5E0ML3XE2KKTLCAIQJVQLEKQ3NAFCYGW1U31FEQ3DRVPIHM2&v=20200309&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [44]:
Venues_List = []
for pc,name,neighbour,lat,lng in zip(DF_TOR['Postcode'],DF_TOR['Borough'],DF_TOR['Neighbourhood'],DF_TOR['Latitude'],DF_TOR['Longitude']):
#     print(pc,name,neighbour,lat,lng)
    url1 = f'https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{lng}&radius={RADIUS}&limit={LIMIT}'
#     print(url1)
    results = requests.get(url1).json()['response']['groups'][0]['items']
    for venue in results:
        Venues_List.append([(pc,
                          name,
                          neighbour,
                          lat,
                          lng,
                          venue['venue']['name'],
                          venue['venue']['location']['lat'],
                          venue['venue']['location']['lng'],
                          venue['venue']['categories'][0]['name'])])
print("FourSqaure api completed")

FourSqaure api completed


In [45]:
A=[]
for items in range(len(Venues_List)):
   A.append(Venues_List[items][0])

In [52]:
A[0:5]

[('M3A',
  'North York',
  'Parkwoods',
  43.7532586,
  -79.3296565,
  'Brookbanks Park',
  43.751976046055574,
  -79.33214044722958,
  'Park'),
 ('M3A',
  'North York',
  'Parkwoods',
  43.7532586,
  -79.3296565,
  'Variety Store',
  43.75197441585782,
  -79.33311418516017,
  'Food & Drink Shop'),
 ('M4A',
  'North York',
  'Victoria Village',
  43.725882299999995,
  -79.31557159999998,
  'Victoria Village Arena',
  43.72348055545508,
  -79.31563520925143,
  'Hockey Arena'),
 ('M4A',
  'North York',
  'Victoria Village',
  43.725882299999995,
  -79.31557159999998,
  'Tim Hortons',
  43.725516631714754,
  -79.31310251366091,
  'Coffee Shop'),
 ('M4A',
  'North York',
  'Victoria Village',
  43.725882299999995,
  -79.31557159999998,
  'Portugril',
  43.72581876267242,
  -79.31278541470671,
  'Portuguese Restaurant')]

In [53]:
Venues_df = pd.DataFrame(item for item in A)

In [54]:
Venues_df.head()

,0,1,2,3,4,5,6,7,8
0,M3A,North York,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,North York,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,North York,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,North York,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,M4A,North York,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [55]:
Venues_df.columns=['Postcode','Borough','Neighbourhood','N_Latitude','N_Longitude','Venue','V_Latitude','V_Longitude','V_Category']

In [56]:
Tor_Venues_df = Venues_df[['Neighbourhood','V_Category']]

In [57]:
Tor_Venue_df_New = pd.get_dummies(Tor_Venues_df,columns=['V_Category'],drop_first=True)

In [58]:
Tor_Venue_df_New = Tor_Venue_df_New.groupby('Neighbourhood').mean()

In [59]:
Tor_Venue_df_New.to_csv("Tornoto_Venue_data.csv")